In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision 
import torchvision.transforms as transforms
from PIL import Image
import os

from customDataset import CustomDataset
from torch.utils.data import DataLoader

from neuralNet import NeuralNet
from VGG import VGGLoss

In [2]:
### Check if GPU is available

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU")
else:
    device = torch.device("cpu")
    print("Using CPU")

# Check for GPU availability and set device
if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    device = torch.device("cuda")
    current_device_idx = torch.cuda.current_device()
else:
    device_count = 0
    device = torch.device("cpu")
    current_device_idx = None

# Print device information
print(f"Number of available GPUs: {device_count}")
print(f"Current device index: {current_device_idx}")
print(f"Current device name: {torch.cuda.get_device_name(current_device_idx)}")

Using GPU
Number of available GPUs: 1
Current device index: 0
Current device name: NVIDIA GeForce RTX 4080


In [3]:
# Dataset creation
# hyperparameters
batch_size = 1
train_X = "LR_train"
train_y = "HR_train"
val_X = "LR_val"
val_y = "HR_val"
test_X = "LR_test"
test_y = "HR_test"


# optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Define a transformation operation to normalize the images
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # normalize to [-1, 1]
])


train_dataset = CustomDataset(train_X, train_y, transform=transforms.ToTensor())
val_dataset = CustomDataset(val_X, val_y, transform=transforms.ToTensor())
test_dataset = CustomDataset(test_X, test_y, transform=transforms.ToTensor())

print("dataset size: ", len(train_dataset))

#### Create the dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

dataset size:  60000


In [4]:
#### Create the model
model = NeuralNet()

# create optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# move them to device (GPU/CPU)
model.to(device)

# loss function
loss_fn1 = VGGLoss(device)
loss_fn2 = nn.MSELoss()

epochs = 100

c:\Users\hrita\anaconda3\envs\dirac\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\hrita\anaconda3\envs\dirac\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
#### Training loop
for epoch in range(epochs):
    print(f"------------------\n Epoch: {epoch}")
    model.train()
    epoch_loss = 0
    batch_loss = 0
    for i, (X, y) in enumerate(train_dataloader):
        if i % 100 == 0:
            print(" Batch:", i, " Loss:", batch_loss/100)
            batch_loss = 0
        
        # retrieve data and move tensors to device
        inputs = X.to(device)
        labels = y.to(device)
        
        # calculate loss
        outputs = model(inputs)
        loss = loss_fn1(outputs, labels)
        loss+= loss_fn2(outputs, labels)
        epoch_loss += loss.item()
        batch_loss += loss.item()
        
        # zero gradient and backpropagate
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # change learning rate
        if epoch/epochs > 0.5:
            optimizer = optim.Adam(model.parameters(), lr=1e-4)
            
        
    # Print epoch information
    print(f"Epoch {epoch+1}/{epochs}: Loss: {epoch_loss:.4f}")
    

------------------
 Epoch: 0
 Batch: 0  Loss: 0.0
 Batch: 100  Loss: 4.688557260036468
 Batch: 200  Loss: 3.7292129468917845
 Batch: 300  Loss: 3.6385039615631105
 Batch: 400  Loss: 3.3535577178001406
 Batch: 500  Loss: 3.334996130466461
 Batch: 600  Loss: 3.550823941230774
 Batch: 700  Loss: 3.0672239577770233
 Batch: 800  Loss: 2.9988957762718202
 Batch: 900  Loss: 3.031031996011734
 Batch: 1000  Loss: 3.003542457818985
 Batch: 1100  Loss: 2.8836509478092194
 Batch: 1200  Loss: 2.9889648365974426
 Batch: 1300  Loss: 2.6362091267108916
 Batch: 1400  Loss: 2.7905691134929658
 Batch: 1500  Loss: 2.535564900636673
 Batch: 1600  Loss: 2.3231813514232638
 Batch: 1700  Loss: 2.3761431550979615
 Batch: 1800  Loss: 2.196658462882042
 Batch: 1900  Loss: 2.2896246725320815
 Batch: 2000  Loss: 2.1790716832876207
 Batch: 2100  Loss: 2.141760392785072
 Batch: 2200  Loss: 2.4221812403202057
 Batch: 2300  Loss: 2.268955955505371


KeyboardInterrupt: 

In [5]:
# Preprocess the input image
image = Image.open("LR_images/nn/downscaled_nn_flickr_cat_000742.jpg")
image.show()
input_image = transform(image)

# # Load the trained model
# model = DeepImagePrior(input_shape)
# model.load_state_dict(torch.load(PATH_TO_SAVED_WEIGHTS))

# Set the model to evaluation mode
model.eval()

# Perform the inference
with torch.no_grad():
    input_image = input_image.to(device)  # Move the input image to the device
    input_image = input_image.unsqueeze(0)  # Add a batch dimension
    output = model(input_image)

# Convert the output to a PIL image
output = output.squeeze(0)
output = transforms.ToPILImage()(output)
output.show()

NameError: name 'transform' is not defined

In [4]:
##############################################
### SAVE MODEL, LOAD MODEL, WORKS ANYWHERE ###
##############################################

model_name = "model.pt"

# Save the model
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, 'checkpoint.pth')

# Load the model
new_model = NeuralNet()
checkpoint = torch.load('checkpoint.pth')
new_model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

print(loss)

NameError: name 'model' is not defined